In [1]:
from IPython.core.display import HTML
from bs4 import BeautifulSoup, UnicodeDammit
import pandas as pd
import StringIO

Objectives:
* Learn how the internet works so we can make it do what we want.
 * www vs. internet
 * GET, PUT, POST, DELETE
 * javascript, APIs.
* Understand basic approach for
 * Simple web scraping
 * Session-based web scraping
 * Crawling?
 * Parsing documents.
* Know that there's a lot more to know, specifically related to
 * Anonymity
 * Selenium

##http vs internet
 * The internet is tubes. 

 * Http is a protocol for describing how documents should look in a web browser.
 * There are other protocols: like git, smtp, ftp etc.

##HTTP can't actually do much.
 * Stateless
 * Dumb framework
 
###Can do
 * GET - think of this as "read"
 * PUT - think of this like an "edit"
 * POST - think of this as "create"
 * DELETE - be careful
 
Everything you do on the web is one of these things.

#Other things



#Scraping a webpage.

 * Use [Requests](http://docs.python-requests.org/en/latest/).
 * Use [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/)
 * Or get fancy and try [lxml](http://lxml.de/)

#I guess I can't scrape this site.
Can you see it?

* Basic auth. 

In [3]:
import requests
z = requests.get('http://galvanizesf.roomzilla.net')


In [6]:
HTML(z.content)

In [7]:
import requests
z = requests.get('http://galvanizesf.roomzilla.net', auth=('', 'gVIP543'))

In [9]:
HTML(z.content)

Fl,Name,Size,Reservations 8am 9am 10am 11am noon 1pm 2pm 3pm 4pm 5pm 6pm 7pm 8pm 9pm 10pm
Fl,Name,Size,8am 9am 10am 11am noon 1pm 2pm 3pm 4pm 5pm 6pm 7pm 8pm 9pm 10pm
1,Alcatraz,6,
2,Bay Bridge,4,
5,Dolores Park,6,Pira - Pira Tritasavit (2:00 pm-2:30 pm)'>
2,Golden Gate Bridge,4,Travis - Carla Holtze (4:00 pm-5:00 pm)'>
5,Irish Hill,6,
3,Mother's/Family Room,1,
5,NOB Hill,4,
3,Ocean Beach,6,Pira - Pira Tritasavit (3:30 pm-4:00 pm)'>
4,Panhandle,8,


In [11]:
#But python can't see it.
z = requests.get('https://accounts.craigslist.org/login/home')
HTML(z.content)

In [ ]:
HTML(z.text)

In [41]:
s = requests.Session()
#z_ = s.get('https://accounts.craigslist.org/login')
clist_pwd = '*******'
form_data = {'step':'confirmation',
         'p': 0,
         'rt': '',
         'rp': '',
         'inputEmailHandle':'isaac.laughlin@gmail.com',
         'inputPassword':clist_pwd}
headers = {"Host": "accounts.craigslist.org",
           "Origin": "https://accounts.craigslist.org",
           "Referer": "https://accounts.craigslist.org/login",
           'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36"}

s.headers.update(headers)
z = s.post('https://accounts.craigslist.org/login', data=form_data)
z

<Response [200]>

<RequestsCookieJar[]>

In [13]:
HTML(z.text)

In [14]:
z.content

'<!DOCTYPE html>\n<html><head>\n    <meta http-equiv="content-type" content="text/html; charset=iso-8859-1">\n\n    <title>craigslist account</title>\n\n    <meta name="description" content="craigslist account">\n    <!--[if lt IE 9]>\n<script src="//www.craigslist.org/js/html5shiv.min.js?v=096822b653643ed1af3136947e4ea79a" type="text/javascript" ></script>\n<![endif]-->\n<!--[if lte IE 7]>\n<script src="//www.craigslist.org/js/json2.min.js?v=178d4ad319e0e0b4a451b15e49b71bec" type="text/javascript" ></script>\n<![endif]-->\n    <link type="text/css" rel="stylesheet" media="all" href="//www.craigslist.org/styles/cl.css?v=94f7e0684eba8acf61bddd1990960a3d">\n    <meta name="viewport" content="initial-scale=1.0, user-scalable=1">\n\n</head>\n\n<body class="account">\n<script type="text/javascript"><!--\n    function C(k){return(document.cookie.match(\'(^|; )\'+k+\'=([^;]*)\')||0)[2]}\n    var pagetype, pagemode;\n    (function(){\n        var h = document.documentElement;\n        h.classN

#Beautiful Soup (or getting data from your webpage).

In [22]:
w = UnicodeDammit(z.content, is_html=True)
soup = BeautifulSoup(z.content, from_encoding='UTF-8')
t = soup.findAll('table')
listings = t[1]

In [32]:
rows = listings.findAll('tr')
data = [[x.text for x in row.findAll('td')] for row in rows]
data

[[],
 [u'\nActive\n',
  u'\n  \n',
  u'\nWood Frame Mirror with Hooks - $50\n    ',
  u'\nsfo\n        furniture - by owner \n    ',
  u'\n        2015-06-23 13:43\n    ',
  u'\n            5088954808\n    '],
 [u'\nActive\n',
  u'\n  \n',
  u'\n2 Tall Yellow Metal Stools - $50\n    ',
  u'\nsfo\n        furniture - by owner \n    ',
  u'\n        2015-06-23 13:43\n    ',
  u'\n            5088953746\n    '],
 [u'\nExpired\n',
  u'\n\n',
  u'\n        2 Tall Yellow Metal Stools - $50\n    ',
  u'\nsfo\n        furniture - by owner \n    ',
  u'\n        2015-06-15 16:35\n    ',
  u'\n            5076242740\n    '],
 [u'\nExpired\n',
  u'\n\n',
  u'\n        Wood Frame Mirror with Hooks - $50\n    ',
  u'\nsfo\n        furniture - by owner \n    ',
  u'\n        2015-06-15 16:33\n    ',
  u'\n            5076240365\n    '],
 [u'\nExpired\n',
  u'\n\n',
  u'\n        Wood Frame Mirror with Hooks - $50\n    ',
  u'\nsfo\n        furniture - by owner \n    ',
  u'\n        2015-05-25 09:39

In [36]:
df = pd.read_html(StringIO.StringIO(t[1]),
                header=0)[0]
df.head()
#WE'RE IN BUSINESS

,status,manage,posting title,area and category,posted date,id
0,Active,NaN,Wood Frame Mirror with Hooks - $50,sfo furniture - by owner,2015-06-23 13:43,5088954808
1,Active,NaN,2 Tall Yellow Metal Stools - $50,sfo furniture - by owner,2015-06-23 13:43,5088953746
2,Expired,NaN,2 Tall Yellow Metal Stools - $50,sfo furniture - by owner,2015-06-15 16:35,5076242740
3,Expired,NaN,Wood Frame Mirror with Hooks - $50,sfo furniture - by owner,2015-06-15 16:33,5076240365
4,Expired,NaN,Wood Frame Mirror with Hooks - $50,sfo furniture - by owner,2015-05-25 09:39,5041686170


#Baseball scores



In [ ]:
scoreboard_html = requests.get('http://m.mlb.com/scoreboard#date=6/25/2015')
HTML(scoreboard_html.content)

#Where's the data?

AJAX. (YAY).

APIs and Javascript.

* A lot of webpages are actually built out of many small APIs (Application Programming Interface). 
  * http://m.mlb.com/scoreboard/
  * http://mlb.mlb.com/gdcross/components/game/mlb/year_2015/month_04/day_19/master_scoreboard.json
* AJAX: the page is updated using javascript.
* This might make scraping hard (requests doesn't resolve javascript)
 * Or easy.

#Logging in to galvanize roomzilla page.

In [ ]:
gz = s.get('https://accounts.craigslist.org/login')

In [ ]:
s.post?

In [44]:
import pymongo


In [45]:
mdb = pymongo.MongoClient('localhost', 27017)

In [55]:
test_db = mdb.test_db

In [56]:
users = test_db.users

In [65]:
users.find_one()

{u'_id': ObjectId('558dcde45dc7bbef09787f8c'),
 u'age': u'37',
 u'friends': [u'David', u'Lyle'],
 u'name': u'Isaac'}

In [66]:
post_id = users.insert({'name':'Boaz'})
post_id

ObjectId('5591baf06291224d1d7afb41')

In [67]:
post_id

ObjectId('5591baf06291224d1d7afb41')

In [68]:
users.find_one({'_id': post_id})

{u'_id': ObjectId('5591baf06291224d1d7afb41'), u'name': u'Boaz'}

In [69]:
users.find_one({'_id': '5591a3866291224d1d7afb40'})

In [70]:
users

Collection(Database(MongoClient('localhost', 27017), u'test_db'), u'users')